In [53]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [54]:
url = "https://www.tomato.gg/tank-stats/NA"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

script = soup.find("script", {"id": "__NEXT_DATA__"})

if script:
    # extract the contents of the script tag
    contents = script.contents[0]
    # parse the JSON data
    data = json.loads(contents)
    # access the array in the initialData property
    initial_data = data["props"]["pageProps"]["data"]
    # print the array
    df = pd.DataFrame(initial_data)
    
else:
    with open('backup.txt', 'r') as f:
        text = f.readlines()

        data = []
        for line in text:
            data.append(eval(line))

        df = pd.DataFrame(data)


print(df)

     tank_id         name   nation  tier class   
0      51361  Progetto 46    Italy     8    MT  \
1      50193   Skorpion G  Germany     8    TD   
2      60225   Bourrasque   France     8    MT   
3       5505   UDES 15/16   Sweden    10    MT   
4      45057     SU-130PM     USSR     8    TD   
..       ...          ...      ...   ...   ...   
555    63761       Pz. 58  Germany     8    MT   
556    53793        T95E2      USA     8    MT   
557    13393       Archer       UK     5    TD   
558    52257       M4A2E4      USA     5    MT   
559    54033   Pz. V/IV A  Germany     5    MT   

                                                 image  battles  winrate   
0    https://api.worldoftanks.com/static/2.66.0/wot...    52480    51.00  \
1    https://api.worldoftanks.com/static/2.66.0/wot...    51868    49.11   
2    https://api.worldoftanks.com/static/2.66.0/wot...    51499    53.30   
3    https://api.worldoftanks.com/static/2.66.0/wot...    51164    49.73   
4    https://api.wo

In [55]:
print(df.columns)

df = df.drop('damage_received_from_invisible', axis=1)
df = df.drop('potential_damage_received', axis=1)
df = df.drop('base_defense_points', axis=1)
df = df.drop('base_capture_points', axis=1)
df = df.drop('image', axis=1)
df = df.drop('sniper_damage', axis=1)
df = df.drop('tracking_assist', axis=1)
df = df.drop('shots_fired', axis=1)
df = df.drop('distance_traveled', axis=1)
df = df.drop('difference', axis=1)
df = df.drop('wn8', axis=1)

print(df.columns)


Index(['tank_id', 'name', 'nation', 'tier', 'class', 'image', 'battles',
       'winrate', 'damage', 'sniper_damage', 'frags', 'spotting_assist',
       'tracking_assist', 'shots_fired', 'direct_hits', 'penetrations',
       'hit_rate', 'pen_rate', 'spots', 'damage_blocked', 'damage_received',
       'damage_received_from_invisible', 'potential_damage_received',
       'base_capture_points', 'base_defense_points', 'life_time', 'survival',
       'distance_traveled', 'wn8', 'difference', 'player_winrate', 'isPrem'],
      dtype='object')
Index(['tank_id', 'name', 'nation', 'tier', 'class', 'battles', 'winrate',
       'damage', 'frags', 'spotting_assist', 'direct_hits', 'penetrations',
       'hit_rate', 'pen_rate', 'spots', 'damage_blocked', 'damage_received',
       'life_time', 'survival', 'player_winrate', 'isPrem'],
      dtype='object')


In [56]:
print(df)

     tank_id         name   nation  tier class  battles  winrate  damage   
0      51361  Progetto 46    Italy     8    MT    52480    51.00    1339  \
1      50193   Skorpion G  Germany     8    TD    51868    49.11    1261   
2      60225   Bourrasque   France     8    MT    51499    53.30    1429   
3       5505   UDES 15/16   Sweden    10    MT    51164    49.73    1880   
4      45057     SU-130PM     USSR     8    TD    50625    49.64    1386   
..       ...          ...      ...   ...   ...      ...      ...     ...   
555    63761       Pz. 58  Germany     8    MT       92    44.47     943   
556    53793        T95E2      USA     8    MT       74    48.73     969   
557    13393       Archer       UK     5    TD       69    52.61     467   
558    52257       M4A2E4      USA     5    MT       48    51.15     394   
559    54033   Pz. V/IV A  Germany     5    MT        2    63.64    1167   

     frags  spotting_assist  ...  penetrations  hit_rate  pen_rate  spots   
0     0.94